In [20]:
from pathlib import Path
import torchtext
torchtext.disable_torchtext_deprecation_warning()

from mltrainer import tokenizer, Trainer, metrics
from mltrainer.rnn_models import NLPmodel, AttentionNLP

import torch
from torch.utils.data import DataLoader
from torch import optim

from mads_datasets import DatasetFactoryProvider, DatasetType
import gin
gin.parse_config_file("imdb.gin")

ParsedConfigFileIncludesAndImports(filename='imdb.gin', imports=[], includes=[])

We load the streamers from the datasetfactory

In [2]:
imdbdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.IMDB)

In [3]:
datasets = imdbdatasetfactory.create_dataset()

2025-02-21 14:26:36.606 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/sarmad/.cache/mads_datasets/imdb
2025-02-21 14:26:36.607 | INFO     | mads_datasets.base:download_data:124 - File already exists at /home/sarmad/.cache/mads_datasets/imdb/aclImdb_v1.tar.gz
2025-02-21 14:26:38.350 | INFO     | mads_datasets.factories.basicfactories:create_dataset:85 - Creating TextDatasets from 25000 trainfilesand 25000 testfiles.
100%|██████████| 25000/25000 [00:00<00:00, 83906.15it/s]


In [4]:
traindataset = datasets["train"]

In [5]:
imdbdatasetfactory.settings

dataset_url: https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
filename: aclImdb_v1.tar.gz
name: imdb
unzip: True
formats: [<FileTypes.TXT: '.txt'>]
digest: 7c2ac02c03563afcf9b574c7e56c153a
maxvocab: 10000
maxtokens: 100
clean_fn: <function clean at 0x716888901800>

In [6]:
from mltrainer.tokenizer import IMDBTokenizer

tokenizer = IMDBTokenizer.fromSettings(
    traindataset=traindataset,
    settings=imdbdatasetfactory.settings
)

2025-02-21 14:26:40.539 | INFO     | mltrainer.tokenizer:build_vocab:120 - Found 79808 tokens


In [7]:
streamers = imdbdatasetfactory.create_datastreamer(batchsize=32, preprocessor=tokenizer)

2025-02-21 14:26:40.651 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/sarmad/.cache/mads_datasets/imdb
2025-02-21 14:26:40.652 | INFO     | mads_datasets.base:download_data:124 - File already exists at /home/sarmad/.cache/mads_datasets/imdb/aclImdb_v1.tar.gz
2025-02-21 14:26:42.421 | INFO     | mads_datasets.factories.basicfactories:create_dataset:85 - Creating TextDatasets from 25000 trainfilesand 25000 testfiles.
100%|██████████| 25000/25000 [00:00<00:00, 81519.28it/s]


In [8]:
train = streamers["train"]
batch = train.batchloop()
tokenizer(batch)

(tensor([[ 142,  161,  145,  ...,   18,   69,   10],
         [ 427, 3414, 7244,  ...,   34,   97,  950],
         [1650,    1,  153,  ...,   12,    4,  323],
         ...,
         [  10,  431,   81,  ...,   27,  138,   84],
         [1308,   22,  460,  ...,   22,   41,  177],
         [  10,   38,    2,  ...,    0,    0,    0]], dtype=torch.int32),
 tensor([0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
         0, 0, 0, 1, 1, 0, 0, 1]))

In [9]:
train = streamers["train"]
print(f"number of batches {len(train)}")
trainstreamer = train.stream()
validstreamer = streamers["valid"].stream()
X, y = next(iter(trainstreamer))
X.shape, y.shape

number of batches 781


(torch.Size([32, 100]), torch.Size([32]))

In [10]:
X

tensor([[  10, 1493,   30,  ...,  190,    9,  149],
        [  10,   25,   75,  ...,    2, 3175, 1448],
        [  93,   93,  139,  ...,    1, 3047,    8],
        ...,
        [  58, 1741, 6676,  ...,    7, 7828,    6],
        [  11,    1,    5,  ...,  217,   42, 1244],
        [  72,   89,   25,  ...,   65,  181,    6]], dtype=torch.int32)

The full dataset has 782 batches of 32 examples

Setup accuracy and loss_fn (this is a classification problem with two classes, 0 and 1)

In [11]:
accuracy = metrics.Accuracy()
loss_fn = torch.nn.CrossEntropyLoss()
log_dir = Path("../../models/nlp/").resolve()


Basic config. We need to specify the vocabulary lenght for the embedding layer.
Trainsteps are set to just 100 batches for speedup in the demo.

In [12]:
from mltrainer import TrainerSettings, ReportTypes

settings = TrainerSettings(
    epochs=10,
    metrics=[accuracy],
    logdir=log_dir,
    train_steps=100,
    valid_steps=25,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.GIN],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
)
settings

2025-02-21 14:26:43.064 | INFO     | mltrainer.settings:check_path:61 - Created logdir /home/sarmad/Documents/code/upperkaam/notebooks_review/Deliverable_Part_3/models/nlp


epochs: 10
metrics: [Accuracy]
logdir: /home/sarmad/Documents/code/upperkaam/notebooks_review/Deliverable_Part_3/models/nlp
train_steps: 100
valid_steps: 25
reporttypes: [<ReportTypes.TENSORBOARD: 2>, <ReportTypes.GIN: 1>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 10}

In [13]:
assert gin.get_bindings("NLPmodel")["config"]["vocab"] == imdbdatasetfactory.settings.maxvocab

In [14]:
model = NLPmodel()
model

NLPmodel(
  (emb): Embedding(10000, 128)
  (rnn): GRU(128, 128, num_layers=3, batch_first=True, dropout=0.1)
  (linear): Linear(in_features=128, out_features=2, bias=True)
)

The base NLP model is just a GRU, with an embedding as a first layer.


In [15]:
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

using cuda


<font color='green'>

**Note:** for now we will run with 'cpu', comment the following code line to run according to available device

**Code added below**

</font>

In [16]:
# comment if you want to run it with cuda (gpu)
device = 'cpu'

In [17]:
optimizer = optim.Adam
scheduler = optim.lr_scheduler.ReduceLROnPlateau

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optimizer,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=scheduler,
    device=device,
    )

2025-02-21 14:26:43.099 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to /home/sarmad/Documents/code/upperkaam/notebooks_review/Deliverable_Part_3/models/nlp/20250221-142643
2025-02-21 14:26:43.698 | INFO     | mltrainer.trainer:__init__:72 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.


In [18]:
trainer.loop()

100%|██████████| 100/100 [00:12<00:00,  8.11it/s]
2025-02-21 14:26:57.035 | INFO     | mltrainer.trainer:report:191 - Epoch 0 train 0.6926 test 0.6888 metric ['0.5513']
100%|██████████| 100/100 [00:11<00:00,  8.55it/s]
2025-02-21 14:27:09.849 | INFO     | mltrainer.trainer:report:191 - Epoch 1 train 0.6889 test 0.6769 metric ['0.5863']
100%|██████████| 100/100 [00:11<00:00,  8.37it/s]
2025-02-21 14:27:22.903 | INFO     | mltrainer.trainer:report:191 - Epoch 2 train 0.6881 test 0.6925 metric ['0.4800']
2025-02-21 14:27:22.903 | INFO     | mltrainer.trainer:__call__:234 - best loss: 0.6769, current loss 0.6925.Counter 1/10.
100%|██████████| 100/100 [00:11<00:00,  8.39it/s]
2025-02-21 14:27:35.942 | INFO     | mltrainer.trainer:report:191 - Epoch 3 train 0.6939 test 0.7124 metric ['0.4900']
2025-02-21 14:27:35.943 | INFO     | mltrainer.trainer:__call__:234 - best loss: 0.6769, current loss 0.7124.Counter 2/10.
100%|██████████| 100/100 [00:11<00:00,  8.48it/s]
2025-02-21 14:27:48.627 | IN

Compare the impact of attention

<font color='green'>

**Answer:** the model with `attension` has faster convergence to minima, better accuracy, and reduced loss due to better long-range dependency capture. 

</font>

In [19]:
attentionmodel = AttentionNLP()

attentiontrainer = Trainer(
    model=attentionmodel,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
    device=device,
    )

attentiontrainer.loop()

2025-02-21 14:28:50.157 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to /home/sarmad/Documents/code/upperkaam/notebooks_review/Deliverable_Part_3/models/nlp/20250221-142850
2025-02-21 14:28:50.158 | INFO     | mltrainer.trainer:__init__:72 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 100/100 [00:11<00:00,  8.86it/s]
2025-02-21 14:29:02.328 | INFO     | mltrainer.trainer:report:191 - Epoch 0 train 0.6744 test 0.6748 metric ['0.5575']
100%|██████████| 100/100 [00:11<00:00,  8.90it/s]
2025-02-21 14:29:14.506 | INFO     | mltrainer.trainer:report:191 - Epoch 1 train 0.6208 test 0.5989 metric ['0.6625']
100%|██████████| 100/100 [00:10<00:00,  9.26it/s]
2025-02-21 14:29:26.190 | INFO     | mltrainer.trainer:report:191 - Epoch 2 train 0.5365 test 0.5282 metric ['0.7362']
100%|██████████| 100/100 [00:11<00:00,  8.83it/s]
2025-02-21 14:29:38.642 | INFO     | mltrainer.trainer:report:191 - Epoch 3 train 0.5041 test 0.4922 metr